# **ConvFinQA**



# **The metric that I will use to evaluate the model will be**
---
## **1.'Answer':** Exact-match: This is the final answer output of the model which tests for accuracy. This can be assessed using a pass(1) or fail (0).
---
---










# 1. Load data & Import Packages

In [1]:
from openai import OpenAI
import json
import os
import re
import random
import pandas as pd
from src.paths import DATA_DIR
from src.models import gpt_4o, gpt_4o_mini

# Ensure dataframe has max with
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 1000)

# Load the API key from environment variables
api_key = os.getenv("OPENAI_API_KEY")
openai = OpenAI(api_key=api_key)

# Training original data
with open(DATA_DIR / "train.json") as file:
    train_orig = json.load(file)

# 2. Understand data

- "pre_text": the texts before the table;

- "post_text": the text after the table;

- "table": the table;

- "id": unique example id. composed by the original report name plus example
index for this report.

- "qa": {
  - "question": the question;

  - "program": the reasoning program;

  - "gold_inds": the gold supporting facts;

  - "exe_ans": the gold execution result;

  - "program_re": the reasoning program in nested format;
}

# 3. Create a filtered data based on chosen keys

In [ ]:
from src.preprocess import to_html, normalize_answer

train_filtered = [
    {
        "id": "".join(file['id']),
        "pre_text": "".join(file["pre_text"]),
        "post_text": "".join(file["post_text"]),
        "table": file["table"],
        "html_table": to_html(file["table"]),
        "program": file['qa']['program'],
        "gold_inds": file['qa']['gold_inds'],
        "question": file["qa"]["question"],
        "gold_answer": file["qa"]["answer"],
        "clean_gold_answer": normalize_answer(file["qa"]["answer"]),

    }
    for file in train_orig
    # Must have 'qa' dict:
    if file.get("qa") is not None # The dictionary file must contain a "qa" key, and the value for "qa" cannot be None
    # 'answer' key must exist and not be blank:
       and file["qa"].get("answer") # Inside the "qa" dictionary, there must be a key "answer" that is not None
       and file["qa"]["answer"].strip() != "" # The "answer" string must not be blank or just whitespace.
    # Must produce a valid float from normalize_answer:
       and normalize_answer(file["qa"]["answer"]) is not None # If normalization fails (e.g., the answer is “abc” or something that doesn’t parse to a float), we exclude that entry.
]

# Understand unique keys
unique_keys = set()
for i in range(len(train_filtered)):
  # Add all top-level keys from this item to our set
  unique_keys.update(train_filtered[i].keys())

unique_keys



{'clean_gold_answer',
 'gold_answer',
 'gold_inds',
 'html_table',
 'id',
 'post_text',
 'pre_text',
 'program',
 'question',
 'table'}

In [ ]:
# import json
# folder_path = "filtered_data/"
# os.makedirs(folder_path, exist_ok=True)  # Ensure the folder exists
# json_file_path = os.path.join(folder_path, "data.json")

# with open(json_file_path, "w", encoding="utf-8") as f:
#     json.dump(train_filtered, f, indent=4)  # Pretty formatting

# print(f"JSON saved at: {json_file_path}")

JSON saved at: filtered_data/data.json


### Understand the kinds of answers (all in string format)

In [121]:
# have a quick look at possible answers
# for i in range(len(train_filtered)):
#   print(train_filtered[i]['answer'])

###  Model Selection and core principles

---
---

**Accuracy target:**

1.  70% of answers need to be an exact match once basic clean up and formatting is accounted for.

2.  70% of program execution needs to be correct.

---
---
Based on OpenAI recomendations I will start with the most capable model I can afford and that permits time, thus gpt-4o.

**Considerations:**
***Cost & Latency *** are considered second to accuracy according to OpenAI. Once a model is found to work for the use-case then swapping with smaller models and using one to a few-shot prompt is recommended. 

# 4. Train(10%) / Dev (10%) / Test(10%) Split 

In [375]:
print("Total records:", len(train_filtered))
print(type(train_filtered))

# get a list of all id's
all_ids = []
for i in range(len(train_filtered)):
  all_ids.append(train_filtered[i]['id'])

# all_ids
num_total = len(all_ids)
num_unique = len(set(all_ids))

# Check they are all unique and therefore make it easy to split
print(num_total)
print(num_unique)

Total records: 2082
<class 'list'>
2082
2082


In [ ]:
import random

# Ensure reproducibility
random.seed(42)  
random.shuffle(train_filtered)

# Total number of records
n = len(train_filtered)

# Correct percentages
train_end = int(0.10 * n) 
dev_end = train_end + int(0.10 * n) 
test_end = dev_end + int(0.10 * n)  

# Assign data splits
train_data = train_filtered[:train_end]  
dev_data   = train_filtered[train_end:dev_end]  
test_data  = train_filtered[dev_end: test_end]  

# Print results
print("TRAIN records:", len(train_data))  
print("DEV records:",   len(dev_data))  
print("TEST records:",  len(test_data))  

print(type(train_data))  
print(type(train_data[0]))  


TRAIN records: 208
DEV records: 208
TEST records: 208
<class 'list'>
<class 'dict'>


In [854]:
train_data

[{'id': 'Single_RE/2015/page_148.pdf-1',
  'pre_text': 'the following table summarized the status of the company 2019s non-vested performance share unit awards and changes for the period indicated : weighted- average grant date performance share unit awards shares fair value .',
  'post_text': '19 .segment reporting the u.s .reinsurance operation writes property and casualty reinsurance and specialty lines of business , including marine , aviation , surety and accident and health ( 201ca&h 201d ) business , on both a treaty and facultative basis , through reinsurance brokers , as well as directly with ceding companies primarily within the u.s .the international operation writes non-u.s .property and casualty reinsurance through everest re 2019s branches in canada and singapore and through offices in brazil , miami and new jersey .the bermuda operation provides reinsurance and insurance to worldwide property and casualty markets through brokers and directly with ceding companies from it

# 5. Experiment 1: Gpt-4o with a basic prompt 

---

## Metric: Exact Match - 'answer'

---

## Rationale: Focused on primarily the correctness of 'answer' and starting with the basics. This is a pass(1) or fail(0).

---

## Control: Baseline Model performance

---

### Step-by-Step Workflow 
- 1. Run experiment via 'process_records' - LLM Model is evaluating 
- 2. Exact Match row level - 'add_local_metric_column' looking for exact match via the 'row_exact_match' function
- 3. Eval at group aggregrate level via the 'exact_match_score'





## Run Experiment  - 1

In [393]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [394]:
from src.answer_prompt.basic import V2_ANSWER_SYSTEM_PROMPT, V1_ANSWER_SYSTEM_PROMPT
from src.llm import openai_llm
from src.preprocess import process_records, construct_main_messages, add_local_metric_column,  add_llm_explanation_column
from src.evals import string_similarity, float_row_exact_match

# 1. 'process_records' creates the dataframe and columns 
train_df = process_records(
    records=train_data,
    system_prompts=V1_ANSWER_SYSTEM_PROMPT,
    model_name=gpt_4o,
    prompt_style="Basic",
    table_key="table",   # or "html_table"
    model_pred_col_name="model_answer" # new column being added from model predictions 
)

train_df.head(10) # approx 2 mins to run 

,id,question,gold_answer,clean_gold_answer,program,table,html_table,model,prompt_style,prompt,model_answer
0,Single_RE/2015/page_148.pdf-1,"what is the total value of granted shares of everest re during 2015 , in millions?",1.9,1.90,"multiply(10705, 178.84), divide(#0, const_1000000)","[[performance share unit awards, year ended december 31 2015 shares, year ended december 31 2015 weighted- average grant date fair value], [outstanding at january 1,, -, $ -], [granted, 10705, 178.84], [vested, -, -], [forfeited, -, -], [outstanding at december 31,, 10705, 178.84]]","<table>\n <thead>\n <tr>\n <th>performance share unit awards</th>\n <th>year ended december 31 2015 shares</th>\n <th>year ended december 31 2015 weighted- average grant date fair value</th>\n </tr>\n </thead>\n <tbody>\n <tr><td>outstanding at january 1,</td><td>-</td><td>$ -</td></tr>\n <tr><td>granted</td><td>10705</td><td>178.84</td></tr>\n <tr><td>vested</td><td>-</td><td>-</td></tr>\n <tr><td>forfeited</td><td>-</td><td>-</td></tr>\n <tr><td>outstanding at december 31,</td><td>10705</td><td>178.84</td></tr>\n </tbody>\n</table>",gpt-4o,Basic,"{'system_prompt': ' You are a financial expert. Your task is to answer the user’s query with a single numeric value in plain text, ensuring the format is easily convertible into a float by the updated `normalize_answer` function. **Follow these rules**: 1. Provide your final answer only as a single numeric value (e.g., 1234.56). 2. Do not include commas (e.g., do not produce “1,234”). 3. Do not include currency symbols (e.g., “$,” “£,” “€,” etc.). 4. Do not include a “%” sign. If the question or data suggest a percentage, still just produce the raw numeric value (e.g., 5% → 5, not 0.05). 5. Do not include trailing letters like K, M, or B. If you need them, remove them (e.g., “1.2M” → “1.2”). 6. Provide no additional commentary or explanation. Return only the numeric answer as a float. No special rounding is required. If a decimal point is appropriate, output it as is. Produce the answer exactly as the numeric value you believe is correct. === PRE TEXT === {pre_text} === POST TEXT === {post_text} === TABLE === {table} '}",1.913272
1,Single_AMT/2010/page_41.pdf-2,what portion of the new sites acquired or constructed during 2010 is located outside united states?,87.9%,87.90,"add(947, 6865), divide(6865, #0)","[[new sites ( acquired or constructed ), 2010, 2009, 2008], [domestic, 947, 528, 160], [international ( 1 ), 6865, 3022, 801]]",<table>\n <thead>\n <tr>\n <th>new sites ( acquired or constructed )</th>\n <th>2010</th>\n <th>2009</th>\n <th>2008</th>\n </tr>\n </thead>\n <tbody>\n <tr><td>domestic</td><td>947</td><td>528</td><td>160</td></tr>\n <tr><td>international ( 1 )</td><td>6865</td><td>3022</td><td>801</td></tr>\n </tbody>\n</table>,gpt-4o,Basic,"{'system_prompt': ' You are a financial expert. Your task is to answer the user’s query with a single numeric value in plain text, ensuring the format is easily convertible into a float by the updated `normalize_answer` function. **Follow these rules**: 1. Provide your final answer only as a single numeric value (e.g., 1234.56). 2. Do not include commas (e.g., do not produce “1,234”). 3. Do not include currency symbols (e.g., “$,” “£,” “€,” etc.). 4. Do not include a “%” sign. If the question or data suggest a percentage, still just produce the raw numeric value (e.g., 5% → 5, not 0.05). 5. Do not include trailing letters like K, M, or B. If you need them, remove them (e.g., “1.2M” → “1.2”). 6. Provide no additional commentary or explanation. Return only the numeric answer as a float. No special rounding is required. If a decimal point is appropriate, output it as is. Produce the answer exactly as the numeric value you believe is correct. === PRE TEXT === {pre_text} === POST TEXT === {post_text} === TABLE === {table} '}",6865
2,Single_UNP/2008/page_34.pdf-2,what was the average other income,108.7,108.70,"add(92, 116), add(#0, 118), divide(#1, const_3)","[[millions of doll

## Experiment 1: Result - Accuracy Exact Match score = 37/208 (17.79%)



---

- Accuracy: 'Exact match score'  
- Utilising a 1 for a pass and 0 or fail. 

---
### Insights

- Some variations of rounding errors e.g 32.5 and pred: 32.53 and others just plain wrong answers. 
- For the former, given finance strict requirements as well as the difference between 0.01, when accumualting and orders of magnitude it can make a difference, thus this is a solid metric.



In [ ]:
from src.evals import float_row_exact_match

# 2. add_local_metric_column - e.g. row_exact_match (row-level analysis) 
#   It creates a new column that you want to eval which you can choose in the param 'new_col_name'

train_df = add_local_metric_column(
    df= train_df,
    metric_func= float_row_exact_match, # Add chosen metric - row level exact match 
    gold_col="clean_gold_answer",
    pred_col="model_answer",
    new_col_name="answer_exact_match" # This is the name of the new column that is created based on the gold vs pred. 
    # Exact match here is just a basic exact string match which is using the row_exact_match function
)
train_df.head()

In [ ]:
from src.evals import exact_match_score

# 3. Eval at group aggregrate level with a function or llm 
print(exact_match_score(train_df, "answer_exact_match"))
train_df.head()

## Experiment 2: Run and Result -  Accuracy Exact Match Score with html table = 42/208 (20.19%)

- Re-run experiment 1 while controlling for html table: Exact same re-run just with html change
- Showcasing an improvement of +5 answer amd 2.4% with just a conversion to html.  
- I will continue with html_table only as input to models moving forwards given that its performing slightly better.

In [ ]:
# 1. 'process_records' creates the dataframe and columns 
train_df = process_records(
    records=train_data,
    system_prompts=V2_ANSWER_SYSTEM_PROMPT,
    model_name=gpt_4o,
    prompt_style="Basic",
    table_key="html_table",
    model_pred_col_name="model_answer" # new column being added from model predictions 
)

train_df.head(2)

In [ ]:
# 2. Create a new column that you want to eval which you can choose in the param 'new_col_name'

train_df = add_local_metric_column(
    df = train_df,
    metric_func= float_row_exact_match, # Add chosen metric - row level exact match 
    gold_col="clean_gold_answer",
    pred_col="model_answer",
    new_col_name="answer_exact_match" # This is the name of the new column that is created based on the gold vs pred. 
    # Exact match here is just a basic exact string match which is using the row_exact_match function
)
# train_df.head(2)

In [ ]:
# 3. Eval at aggreate level to see performance
print(exact_match_score(train_df, "answer_exact_match"))
train_df.head(10)

# 6. LLM as Judge - Let's use insights of model to improve prompts only looking at where the model answers wrong

In [506]:
# Lets only look at false 
train_df_false_only = train_df[train_df["answer_exact_match"] == False]
len(train_df_false_only)

166

In [ ]:
from src.LLM_as_judge.answer import V1_EXPLANATION_ANSWER_PROMPT

llm_exp_train_df = add_llm_explanation_column(
    df=train_df_false_only,
    system_prompts_dict=V1_EXPLANATION_ANSWER_PROMPT,
    model_name=gpt_4o,
    gold_col="clean_gold_answer",
    pred_col="model_answer",
    new_col_name="answer_reasoning",
    table = "html_table"
)

llm_exp_train_df.head(5)

In [ ]:
llm_exp_train_df['answer_reasoning'].to_list()[:10]

# 6.1 Ask LLM to put Errors into categories

In [ ]:
from src.LLM_as_judge.answer import CATEGORISE_ANSWER_PROMPT

llm_exp_train_df= add_llm_explanation_column(
    df=llm_exp_train_df,
    system_prompts_dict=CATEGORISE_ANSWER_PROMPT,
    model_name=gpt_4o,
    gold_col="clean_gold_answer",
    pred_col="answer_reasoning",
    new_col_name="answer_error_category",
    table = "html_table"
)

llm_exp_train_df.head(5)

# 6.2 Looking at error categories
--- 
## - Rounding Errors: 65.1% 
## - Different_data_or_major_difference: 30.1%
## - Sign error: 4.8%

## **Open Question**: Can I get LLM to improve providing this extra information in the prompts?

In [ ]:
llm_exp_train_df['answer_error_category'] = llm_exp_train_df['answer_error_category'].astype('category')
print(llm_exp_train_df['answer_error_category'].dtypes)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


category_counts = llm_exp_train_df['answer_error_category'].value_counts(dropna=False)
print(category_counts)

plt.figure(figsize=(5, 5))
category_counts.plot(kind='pie', autopct='%1.1f%%')
plt.title('Error Category Distribution')
plt.ylabel('')  # hide the default label
plt.show()


In [ ]:
sampled_df = (
    llm_exp_train_df
    .groupby("answer_error_category", observed=False)  # or observed=True, as you prefer
    .head(15)  # Up to 5 rows per category
    .loc[:, ["question", "answer_error_category", "gold_answer", "clean_gold_answer", "model_answer"]]
)

# Sort by the category column so rows are grouped visually
sampled_df = sampled_df.sort_values(by="answer_error_category")

sampled_df

,question,answer_error_category,gold_answer,clean_gold_answer,model_answer
11,what percent of system energy's receivable from the money pool was replaced by a note receivable from entergy new orleans?,DIFFERENT_DATA_OR_MAJOR_DIFFERENCE,42%,42.00,53.33
1,what portion of the new sites acquired or constructed during 2010 is located outside united states?,DIFFERENT_DATA_OR_MAJOR_DIFFERENCE,87.9%,87.90,6865.0
67,what was the total amount of corporate and other expenses from 2015-2017?,DIFFERENT_DATA_OR_MAJOR_DIFFERENCE,415.6,415.60,400.4
65,"if vies were consolidated , what would the total minimum lease payments increase to , in millions?",DIFFERENT_DATA_OR_MAJOR_DIFFERENCE,10282,10282.00,5801
64,what is the total net operating loss carryforwards?,DIFFERENT_DATA_OR_MAJOR_DIFFERENCE,2484034,2484034.00,2384034.00
5,what is the total return for every dollar invested in advanced auto parts in january 2009 and sold in january 2011?,DIFFERENT_DATA_OR_MAJOR_DIFFERENCE,0.96,0.96,1.96
63,for the= quarter ended march 31 what was the percent of the change in the stock price from the highest to the lowest,DIFFERENT_DATA_OR_MAJOR_DIFFERENCE,9.3%,9.30,8.50
10,"for acquired customer-related and network location intangibles , what is the expected annual amortization expenses , in millions?",DIFFERENT_DATA_OR_MAJOR_DIFFERENCE,7.4,7.40,1.06
27,what is the percentage difference in the number of shares to be issued if the stock price closes at $ 11 compared to if it closes at $ 20?,DIFFERENT_DATA_OR_MAJOR_DIFFERENCE,278%,278.00,2277.78
55,what was the change in percentage of consolidated net sales from 2006 to 2008?,DIFFERENT_DATA_OR_MAJOR_DIFFERENCE,73%,73.00,10.06


# 7. Allowing for rounding Errors for 1e-2 (0.01): 54/208 (25.96%) 
## However, I won't use this as its a financial task which would require higher levels of accuracy

In [ ]:
# 2. It creates a new column that you want to eval which you can choose in the param 'new_col_name'

from src.evals import is_within_tolerance

train_df_fault_tolerance = add_local_metric_column(
    df= train_df,
    metric_func= is_within_tolerance, # Set to 1e-2 (0.01)
    gold_col="clean_gold_answer",
    pred_col="model_answer",
    new_col_name="answer_fault_tolerance" # This is the name of the new column that is created based on the gold vs pred. 
    # Exact match here is just a basic exact string match which is using the row_exact_match function
)
train_df_fault_tolerance.head(2)

,id,question,gold_answer,clean_gold_answer,program,table,html_table,model,prompt_style,prompt,model_answer,answer_exact_match,answer_fault_tolerance
0,Single_RE/2015/page_148.pdf-1,"what is the total value of granted shares of everest re during 2015 , in millions?",1.9,1.9,"multiply(10705, 178.84), divide(#0, const_1000000)","[[performance share unit awards, year ended december 31 2015 shares, year ended december 31 2015 weighted- average grant date fair value], [outstanding at january 1,, -, $ -], [granted, 10705, 178.84], [vested, -, -], [forfeited, -, -], [outstanding at december 31,, 10705, 178.84]]","<table>\n <thead>\n <tr>\n <th>performance share unit awards</th>\n <th>year ended december 31 2015 shares</th>\n <th>year ended december 31 2015 weighted- average grant date fair value</th>\n </tr>\n </thead>\n <tbody>\n <tr><td>outstanding at january 1,</td><td>-</td><td>$ -</td></tr>\n <tr><td>granted</td><td>10705</td><td>178.84</td></tr>\n <tr><td>vested</td><td>-</td><td>-</td></tr>\n <tr><td>forfeited</td><td>-</td><td>-</td></tr>\n <tr><td>outstanding at december 31,</td><td>10705</td><td>178.84</td></tr>\n </tbody>\n</table>",gpt-4o,Basic,"{'system_prompt': ' You are a financial expert. Your task is to answer the user’s query with a single numeric value in plain text, ensuring the format is easily convertible into a float by the updated `normalize_answer` function. **Follow these rules**: 1. Provide your final answer only as a single numeric value (e.g., 1234.56). 2. Do not include commas (e.g., do not produce “1,234”). 3. Do not include currency symbols (e.g., “$,” “£,” “€,” etc.). 4. Do not include a “%” sign. If the question or data suggest a percentage, still just produce the raw numeric value (e.g., 5% → 5, not 0.05). 5. Do not include trailing letters like K, M, or B. If you need them, remove them (e.g., “1.2M” → “1.2”). 6. Round your answer to 2 decimal places where appropriat. 6. Provide no additional commentary or explanation. Return only the numeric answer as a float. No special rounding is required. If a decimal point is appropriate, output it as is. Produce the answer exactly as the numeric value you believe is correct. === PRE TEXT === {pre_text} === POST TEXT === {post_text} === HTML TABLE === {table} '}",1.91,False,False
1,Single_AMT/2010/page_41.pdf-2,what portion of the new sites acquired or constructed during 2010 is located outside united states?,87.9%,87.9,"add(947, 6865), divide(6865, #0)","[[new sites ( acquired or constructed ), 2010, 2009, 2008], [domestic, 947, 528, 160], [international ( 1 ), 6865, 3022, 801]]",<table>\n <thead>\n <tr>\n <th>new sites ( acquired or constructed )</th>\n <th>2010</th>\n <th>2009</th>\n <th>2008</th>\n </tr>\n </thead>\n <tbody>\n <tr><td>domestic</td><td>947</td><td>528</td><td>160</td></tr>\n <tr><td>international ( 1 )</td><td>6865</td><td>3022</td><td>801</td></tr>\n </tbody>\n</table>,gpt-4o,Basic,"{'system_prompt': ' You are a financial expert. Your task is to answer the user’s query with a single numeric value in plain text, ensuring the format is easily convertible into a float by the updated `normalize_answer` function. **Follow these rules**: 1. Provide your final answer only as a single numeric value (e.g., 1234.56). 2. Do not include commas (e.g., do not produce “1,234”). 3. Do not include currency symbols (e.g., “$,” “£,” “€,” etc.). 4. Do not include a “%” sign. If the question or data suggest a percentage, still just produce the raw numeric value (e.g., 5% → 5, not 0.05). 5. Do not include trailing letters like K, M, or B. If you need them, remove them (e.g., “1.2M” → “1.2”). 6. Round your answer to 2 decimal places where appropriat. 6. Provide no additional commentary or explanation. Return only the numeric answer as a float. No special rounding is required. If a decimal point is appropriate, output it as is. Produce the answer exactly as the numeric value you believe is correct. === PRE TEXT === {pre_text} === POST TEXT === {post_text} === HTML

In [796]:
# 3. Eval at aggreate level to see performance
print(exact_match_score(train_df_fault_tolerance , "answer_fault_tolerance"))
train_df_fault_tolerance.head(2)

Accuracy Score: 54/208 (25.96%)


,id,question,gold_answer,clean_gold_answer,program,table,html_table,model,prompt_style,prompt,model_answer,answer_exact_match,answer_fault_tolerance
0,Single_RE/2015/page_148.pdf-1,"what is the total value of granted shares of everest re during 2015 , in millions?",1.9,1.9,"multiply(10705, 178.84), divide(#0, const_1000000)","[[performance share unit awards, year ended december 31 2015 shares, year ended december 31 2015 weighted- average grant date fair value], [outstanding at january 1,, -, $ -], [granted, 10705, 178.84], [vested, -, -], [forfeited, -, -], [outstanding at december 31,, 10705, 178.84]]","<table>\n <thead>\n <tr>\n <th>performance share unit awards</th>\n <th>year ended december 31 2015 shares</th>\n <th>year ended december 31 2015 weighted- average grant date fair value</th>\n </tr>\n </thead>\n <tbody>\n <tr><td>outstanding at january 1,</td><td>-</td><td>$ -</td></tr>\n <tr><td>granted</td><td>10705</td><td>178.84</td></tr>\n <tr><td>vested</td><td>-</td><td>-</td></tr>\n <tr><td>forfeited</td><td>-</td><td>-</td></tr>\n <tr><td>outstanding at december 31,</td><td>10705</td><td>178.84</td></tr>\n </tbody>\n</table>",gpt-4o,Basic,"{'system_prompt': ' You are a financial expert. Your task is to answer the user’s query with a single numeric value in plain text, ensuring the format is easily convertible into a float by the updated `normalize_answer` function. **Follow these rules**: 1. Provide your final answer only as a single numeric value (e.g., 1234.56). 2. Do not include commas (e.g., do not produce “1,234”). 3. Do not include currency symbols (e.g., “$,” “£,” “€,” etc.). 4. Do not include a “%” sign. If the question or data suggest a percentage, still just produce the raw numeric value (e.g., 5% → 5, not 0.05). 5. Do not include trailing letters like K, M, or B. If you need them, remove them (e.g., “1.2M” → “1.2”). 6. Round your answer to 2 decimal places where appropriat. 6. Provide no additional commentary or explanation. Return only the numeric answer as a float. No special rounding is required. If a decimal point is appropriate, output it as is. Produce the answer exactly as the numeric value you believe is correct. === PRE TEXT === {pre_text} === POST TEXT === {post_text} === HTML TABLE === {table} '}",1.91,False,False
1,Single_AMT/2010/page_41.pdf-2,what portion of the new sites acquired or constructed during 2010 is located outside united states?,87.9%,87.9,"add(947, 6865), divide(6865, #0)","[[new sites ( acquired or constructed ), 2010, 2009, 2008], [domestic, 947, 528, 160], [international ( 1 ), 6865, 3022, 801]]",<table>\n <thead>\n <tr>\n <th>new sites ( acquired or constructed )</th>\n <th>2010</th>\n <th>2009</th>\n <th>2008</th>\n </tr>\n </thead>\n <tbody>\n <tr><td>domestic</td><td>947</td><td>528</td><td>160</td></tr>\n <tr><td>international ( 1 )</td><td>6865</td><td>3022</td><td>801</td></tr>\n </tbody>\n</table>,gpt-4o,Basic,"{'system_prompt': ' You are a financial expert. Your task is to answer the user’s query with a single numeric value in plain text, ensuring the format is easily convertible into a float by the updated `normalize_answer` function. **Follow these rules**: 1. Provide your final answer only as a single numeric value (e.g., 1234.56). 2. Do not include commas (e.g., do not produce “1,234”). 3. Do not include currency symbols (e.g., “$,” “£,” “€,” etc.). 4. Do not include a “%” sign. If the question or data suggest a percentage, still just produce the raw numeric value (e.g., 5% → 5, not 0.05). 5. Do not include trailing letters like K, M, or B. If you need them, remove them (e.g., “1.2M” → “1.2”). 6. Round your answer to 2 decimal places where appropriat. 6. Provide no additional commentary or explanation. Return only the numeric answer as a float. No special rounding is required. If a decimal point is appropriate, output it as is. Produce the answer exactly as the numeric value you believe is correct. === PRE TEXT === {pre_text} === POST TEXT === {post_text} === HTML

# 8. Few-shot Prompt: Exact Match Accuracy 39/208 (18.75%)

In [718]:
from src.answer_prompt.few_shot import V1_ANSWER_FEW_SHOT_PROMPT

# 1. 'process_records' creates the dataframe and columns 
few_shot_prompt_train_df = process_records(
    records=train_data,
    system_prompts=V1_ANSWER_FEW_SHOT_PROMPT,
    model_name=gpt_4o,
    prompt_style="few-shot",
    table_key="html_table",
    model_pred_col_name="model_answer" # new column being added from model predictions 
)

In [ ]:
# 2. Creates a new column that you want to eval which you can choose in the param 'new_col_name'

few_shot_prompt_train_df = add_local_metric_column(
    df=few_shot_prompt_train_df,
    metric_func=float_row_exact_match, # Add chosen metric - row level exact match 
    gold_col="clean_gold_answer",
    pred_col="model_answer",
    new_col_name="answer_exact_match" # This is the name of the new column that is created based on the gold vs pred. 
    # Exact match here is just a basic exact string match which is using the row_exact_match function
)

few_shot_prompt_train_df.head(2)

In [ ]:
# 3. Eval at aggreate level to see performance
print(exact_match_score(few_shot_prompt_train_df, "answer_exact_match"))
few_shot_prompt_train_df.head(2)

# 9. Hidden CoT - Accuracy Exact Match Score: 40/208 (19.23%)

In [ ]:
from src.answer_prompt.CoT_style import  V2_COT_ANSWER_SYSTEM_PROMPT

# 1. 'process_records' creates the dataframe and columns 
train_df_exp3 = process_records(
    records=train_data,
    system_prompts=V2_COT_ANSWER_SYSTEM_PROMPT,
    model_name=gpt_4o,
    prompt_style="CoT",
    table_key="html_table",
    model_pred_col_name="model_answer" # new column being added from model predictions 
)

train_df_exp3.head(2)

In [ ]:
# 2. Creates a new column that you want to eval which you can choose in the param 'new_col_name'

train_df_exp3 = add_local_metric_column(
    df= train_df_exp3,
    metric_func=float_row_exact_match, # Add chosen metric - row level exact match 
    gold_col="clean_gold_answer",
    pred_col="model_answer",
    new_col_name="answer_exact_match" # This is the name of the new column that is created based on the gold vs pred. 
    # Exact match here is just a basic exact string match which is using the row_exact_match function
)

train_df_exp3.head(2)

In [ ]:
# 3. Eval at aggreate level to see performance

# V2 system prompt 
print(exact_match_score(train_df_exp3, "answer_exact_match"))
train_df_exp3.head(2)

# 9. Dev set testing - I'm not able to get the model to perform well enough on the train set to start testing on dev. 

- Once I get training set to perform better I would, then start testin against dev, make futher adaptations based on error analysis, perhaps different pre-processing and other prompt engineering techniques. 
- Once the llm is performing well on the dev set I would finally test on test set 

# 10. Results & Insights

# - Original prompts: 42/208 (20.19%)

# - Fault tolerance of 1e-2: 54/208 (25.96%)


# - Few-shot: 39/208 (18.75%)

# - Hidden Cot: 40/208 (19.23%)


---
## Metrics established for the exact match of 'answer' using Gpt-4o:
---
- We have baseline metrics utilising gpt-4o.
1. html table vs non-html table - html showed a unexepectedly increase of +2.4% on predictions.

---
---

## Insights 
- I utilised these incorrect answers to look at where the categories of errors were coming from to guide in creating better prompts or perhaps pre-processing.
- Given that 65.1% were coming from rounding errors I tried to inform the prompts, however, there was not any immediate improvements made. 
- The original first prompt that I used seemed to have performed the best (20.19%), follwed by Hidden CoT (19.23%) and then Few-Shot (18.75%) exact match accuracy.


## Recommendations
- Utilise as reasoning model
- Fruther prompt optimization 
- Fine-tuning 


# Save data as Csv

In [ ]:
# data_folder_path = "answer_data/"
# os.makedirs(data_folder_path, exist_ok=True)

# file_path = os.path.join(data_folder_path,'train_df.csv')
# train_df.to_csv(file_path, index = False)

In [ ]:
# file_path = os.path.join(data_folder_path,'llm_exp_train_df.csv')
# llm_exp_train_df.to_csv(file_path, index=False)

In [ ]:
# file_path = os.path.join(data_folder_path,'train_df_fault_tolerance.csv')
# train_df_fault_tolerance.to_csv(file_path, index=False)

In [ ]:
# file_path = os.path.join(data_folder_path,'few_shot.csv')
# few_shot_prompt_train_df.to_csv(file_path, index=False)

In [ ]:
# file_path = os.path.join(data_folder_path,'Cot_prompt.csv')
# train_df_exp3.to_csv(file_path, index=False)